In [4]:
import os

In [2]:
%pwd

'c:\\NLP PROJECTS\\Text-Summarization-NLP-Project\\research'

In [3]:
os.chdir("../")

In [5]:
%pwd

'c:\\NLP PROJECTS\\Text-Summarization-NLP-Project'

In [13]:
import os
import urllib.request as request
import zipfile
from pathlib import Path
from dataclasses import dataclass
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml, create_directories, get_size
from textSummarizer.logging import logger

# Data Class for Data Ingestion Configuration
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Configuration Manager Class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=Path(CONFIG_FILE_PATH),  # Convert to Path
        params_filepath=Path(PARAMS_FILE_PATH)   # Convert to Path
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Convert paths to Path objects
        root_dir = Path(config.root_dir)
        local_data_file = Path(config.local_data_file)
        unzip_dir = Path(config.unzip_dir)

        create_directories([root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=root_dir,
            source_URL=config.source_URL,
            local_data_file=local_data_file,
            unzip_dir=unzip_dir
        )

        return data_ingestion_config

# Data Ingestion Class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not self.config.local_data_file.exists():
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=str(self.config.local_data_file)  # Convert Path to string
            )
            logger.info(f"{filename} downloaded! Info:\n{headers}")
        else:
            logger.info(f"File already exists. Size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        """Extracts the zip file into the data directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")

# Running the Code with Exception Handling
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise RuntimeError(f"Error in data ingestion: {e}")


[2025-03-22 17:28:25,069: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-22 17:28:25,078: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-22 17:28:25,080: INFO: common: created directory at: artifacts]
[2025-03-22 17:28:25,082: INFO: common: created directory at: artifacts\data_ingestion]
[2025-03-22 17:28:33,520: INFO: 1764291647: artifacts\data_ingestion\data.zip downloaded! Info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 1549:1BE757:27349A:490AC6:67DE9EBF
Accept-Ranges: bytes
Date: Sat, 22 Mar 2025 11:58:29 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4723-BOM
X-Cache: HIT
X-Cache-Hits